In [2]:
import os
import fitz
from dotenv import load_dotenv
from langchain.schema import Document
import chromadb
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from sklearn.feature_extraction.text import TfidfVectorizer


# Step 0: Load environment and set paths
load_dotenv()

# Detect root folder (works even if running from /notebooks)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "..")) \
    if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()

DATA_DIR = os.path.join(ROOT_DIR, "data")
CHROMA_DIR = os.path.join(ROOT_DIR, "chroma_db")
EMBED_MODEL = "BAAI/bge-large-en-v1.5"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CHROMA_DIR, exist_ok=True)

print(f"📂 Root dir: {ROOT_DIR}")
print(f"📁 Data dir: {DATA_DIR}")
print(f"💾 Chroma dir: {CHROMA_DIR}")

# Step 1: Load text from PDFs
def load_pdfs_from_folder(folder_path):
    all_docs = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            pdf = fitz.open(file_path)
            for page_num, page in enumerate(pdf, start=1):
                text = page.get_text("text") or ""
                if text.strip():
                    all_docs.append(
                        Document(
                            page_content=text.strip(),
                            metadata={"source": file, "page": page_num}
                        )
                    )
            pdf.close()
    return all_docs

docs = load_pdfs_from_folder(DATA_DIR)
print(f"✅ Loaded {len(docs)} pages from PDFs.")

if not docs:
    raise ValueError("❌ No PDF pages found! Please add PDFs inside the /data folder.")

/home/jagannath/myPocketLawyer/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📂 Root dir: /home/jagannath/myPocketLawyer
📁 Data dir: /home/jagannath/myPocketLawyer/data
💾 Chroma dir: /home/jagannath/myPocketLawyer/chroma_db
✅ Loaded 484 pages from PDFs.


In [55]:
# Diverse queries for Nepal law
queries = [
    "When was the Constitution of Nepal officially published?",
    "What is the Constitutional rule regarding the death penalty in Nepal?",
    "How is a 'heinous offence' defined in Nepal's criminal law?",
    "What is the minimum age for a child to be held criminally?",
    "Can a person who is now a foreign citizen still get a special type of Nepalese citizenship, and if so, what rights do they enjoy?",
    "Is gambling legal in Nepal?",
    "What fundamental rights are guaranteed by the Constitution of Nepal?",
    "Who does the government appoint to serve as a Probation Officer or Parole Officer?"
]

Expected_answers = [
    "The Constitution of Nepal was officially published on September 20, 2015 (2072.6.3).",
    "The Constitution strictly prohibits the making of any law that provides for the death penalty.",
    "A heinous offence is an act punishable by life imprisonment or a prison term of more than ten years.",
    "An act is not considered an offence if it is committed by a child who is below ten years of age.",
    "Yes, non-resident Nepalese citizenship can be granted to individuals who or whose ancestors were Nepali citizens",
    "gambling is treated as an offense against public order and is illegal.",
    "The Constitution guarantees a comprehensive list of fundamental rights, including the Right to Live with Dignity, Right to Freedom etc.",
    "The Government of Nepal may appoint as a Probation Officer or Parole Officer.",

]

In [4]:
# Example: TF-IDF Retriever

# Convert Document objects to plain text for TF-IDF
doc_texts = [doc.page_content for doc in docs]


# Fit the TF-IDF vectorizer on the text content

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(doc_texts)


In [5]:
# TF-IDF retriever

def tfidf_retrieve(query, top_k=3):
    query_vec = vectorizer.transform([query])
    scores = (tfidf_matrix * query_vec.T).toarray().ravel()
    top_indices = scores.argsort()[::-1][:top_k]
    return [doc_texts[i] for i in top_indices]



# Evaluate TF-IDF Retriever

for i, query in enumerate(queries):
    results = tfidf_retrieve(query)
    print(f"Query {i+1}: {query}")
    print("Top Result:", results[0][:100], "...\n")
    print("Expected:", Expected_answers[i], "\n")

Query 1: Is bomb illegal in nepal?
Top Result: 12 
(3) 
The right of private defence under this Section shall be 
exercised only when there is a re ...

Expected: Yes, the bomb is illegal in Nepal 

Query 2: What happens if you kill someone in Nepal?
Top Result: 44 
Chapter-2 
Offences against Public Tranquility 
60. 
Prohibition of unlawful assembly: (1) No pe ...

Expected: You will be arrested 

Query 3: Is children are allow to work in Nepal?
Top Result: 10 
 
Part-2 
Citizenship 
10. 
Not to deprive of citizenship: (1) No citizen of Nepal may be depriv ...

Expected: No, children is not allowed to work. 



In [1]:

# # Dense Retriever (Sentence Transformers)
# # We will use a dense embedding model to retrieve relevant documents.
# model = SentenceTransformer('all-MiniLM-L6-v2')
# doc_embeddings = model.encode(doc_texts)


In [2]:
# import numpy as np

# def dense_retrieve(query, top_k=3):
#     query_emb = model.encode([query])[0]
#     scores = np.dot(doc_embeddings, query_emb)
#     top_indices = scores.argsort()[::-1][:top_k]
#     return [doc_texts[i] for i in top_indices]

# # Evaluate Dense Retriever
# for i, query in enumerate(queries):
#     results = dense_retrieve(query)
#     print(f"Query {i+1}: {query}")
#     print("Top Result:", results[0][:200], "...\n")
#     print("Expected:", Expected_answers[i], "\n")

In [61]:
# Semantic Retriever using Sentence Transformers

semantic_model = SentenceTransformer('all-MiniLM-L6-v2') 
semantic_embeddings = semantic_model.encode(doc_texts, convert_to_tensor=True)



In [62]:
def semantic_retrieve(query, top_k=3): 
    query_emb = semantic_model.encode(query, convert_to_tensor=True) 
    scores = util.pytorch_cos_sim(query_emb, semantic_embeddings)[0] 
    top_indices = scores.argsort(descending=True)[:top_k] 
    return [doc_texts[i] for i in top_indices]

# Evaluate Dense Retriever
for i, query in enumerate(queries): 
    results = semantic_retrieve(query) 
    print(f"Query {i+1}: {query}") 
    print("Top Semantic Result:", results[0][:300], "...\n") 
    print("Expected:", Expected_answers[i], "\n") 



Query 1: When was the Constitution of Nepal officially published?
Top Semantic Result: 1 
 
 
 
 
 
THE CONSTITUTION OF NEPAL ...

Expected: The Constitution of Nepal was officially published on September 20, 2015 (2072.6.3). 

Query 2: What is the Constitutional rule regarding the death penalty in Nepal?
Top Semantic Result: 1 
The Criminal Offences (Sentencing and 
Execution) Act, 2017 (2074)  
 
Date of Authentication:  
16 October 2017 (2074.6.30) 
 
Act Number 38 of the year 2017 (2074) 
An Act Made to Provide for Determination and Execution of 
Sentences for Criminal Offences 
 
Preamble: Whereas it is expedient to ...

Expected: The Constitution strictly prohibits the making of any law that provides for the death penalty. 

Query 3: How is a 'heinous offence' defined in Nepal's criminal law?
Top Semantic Result: 1 
The Criminal Offences (Sentencing and 
Execution) Act, 2017 (2074)  
 
Date of Authentication:  
16 October 2017 (2074.6.30) 
 
Act Number 38 of the year 2017 (2074) 

In [63]:
# Hybrid Retriever: Combine TF-IDF and Semantic scores

def hybrid_retrieve(query, top_k=3, alpha=0.5): 
    # TF-IDF scores 
    query_vec = vectorizer.transform([query]) 
    tfidf_scores = (tfidf_matrix * query_vec.T).toarray().ravel() 
    tfidf_scores = tfidf_scores / (np.max(tfidf_scores) + 1e-8) # Normalize

    # Semantic scores
    query_emb = semantic_model.encode(query, convert_to_tensor=True)
    semantic_scores = util.pytorch_cos_sim(query_emb, semantic_embeddings)[0].cpu().numpy()
    semantic_scores = semantic_scores / (np.max(semantic_scores) + 1e-8)  # Normalize


    # Combine scores
    hybrid_scores = alpha * tfidf_scores + (1 - alpha) * semantic_scores
    top_indices = hybrid_scores.argsort()[::-1][:top_k]
    return [doc_texts[i] for i in top_indices]


In [64]:
# Evaluate Hybrid Retriever
for i, query in enumerate(queries): 
    results = hybrid_retrieve(query) 
    print(f"Query {i+1}: {query}") 
    print("Top Hybrid Result:", results[0][:200], "...\n") 
    print("Expected:", Expected_answers[i], "\n") 


Query 1: When was the Constitution of Nepal officially published?
Top Hybrid Result: 1 
 
 
 
 
 
THE CONSTITUTION OF NEPAL ...

Expected: The Constitution of Nepal was officially published on September 20, 2015 (2072.6.3). 

Query 2: What is the Constitutional rule regarding the death penalty in Nepal?
Top Hybrid Result: 1 
 
 
 
 
 
THE CONSTITUTION OF NEPAL ...

Expected: The Constitution strictly prohibits the making of any law that provides for the death penalty. 

Query 3: How is a 'heinous offence' defined in Nepal's criminal law?
Top Hybrid Result: 17 
Chapter -3 
Criminal Conspiracy, Attempt, Abetment and Accomplice 
33. Prohibition of criminal conspiracy: (1) No person shall make a 
criminal conspiracy. 
(2) 
Where two or more persons agree to ...

Expected: A heinous offence is an act punishable by life imprisonment or a prison term of more than ten years. 

Query 4: What is the minimum age for a child to be held criminally?
Top Hybrid Result: 11 
(c) 
for a recidivist, a se

In [ ]:
# putting it all together

class retreiver_evaluation:
    """
    Evaluate TF-IDF, Dense, Semantic, and Hybrid retrievers on legal document data.
    """

    def __init__(self, docs, queries, expected_answers):
        # Convert Document objects to plain text
        self.doc_texts = [doc.page_content for doc in docs]
        self.queries = queries
        self.expected_answers = expected_answers

        # TF-IDF setup
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(self.doc_texts)

        # Dense/Semantic setup
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.doc_embeddings = self.model.encode(self.doc_texts)
        self.semantic_model = self.model
        self.semantic_embeddings = self.semantic_model.encode(self.doc_texts, convert_to_tensor=True)

    def tfidf_retrieve(self, query, top_k=3):
        query_vec = self.vectorizer.transform([query])
        scores = (self.tfidf_matrix * query_vec.T).toarray().ravel()
        top_indices = scores.argsort()[::-1][:top_k]
        return [self.doc_texts[i] for i in top_indices]


    def semantic_retrieve(self, query, top_k=3):
        from sentence_transformers import util
        query_emb = self.semantic_model.encode(query, convert_to_tensor=True)
        scores = util.pytorch_cos_sim(query_emb, self.semantic_embeddings)[0]
        top_indices = scores.argsort(descending=True)[:top_k]
        return [self.doc_texts[i] for i in top_indices]

    def hybrid_retrieve(self, query, top_k=3, alpha=0.5):
        from sentence_transformers import util
        import numpy as np
        # TF-IDF scores
        query_vec = self.vectorizer.transform([query])
        tfidf_scores = (self.tfidf_matrix * query_vec.T).toarray().ravel()
        tfidf_scores = tfidf_scores / (np.max(tfidf_scores) + 1e-8)
        # Semantic scores
        query_emb = self.semantic_model.encode(query, convert_to_tensor=True)
        semantic_scores = util.pytorch_cos_sim(query_emb, self.semantic_embeddings)[0].cpu().numpy()
        semantic_scores = semantic_scores / (np.max(semantic_scores) + 1e-8)
        # Combine scores
        hybrid_scores = alpha * tfidf_scores + (1 - alpha) * semantic_scores
        top_indices = hybrid_scores.argsort()[::-1][:top_k]
        return [self.doc_texts[i] for i in top_indices]

    def evaluate(self):
        print("=== TF-IDF Retriever ===")
        for i, query in enumerate(self.queries):
            results = self.tfidf_retrieve(query)
            print(f"Query {i+1}: {query}")
            print("Top Result:", results[0][:100], "...\n")
            print("Expected:", self.expected_answers[i], "\n")


        print("=== Semantic Retriever ===")
        for i, query in enumerate(self.queries):
            results = self.semantic_retrieve(query)
            print(f"Query {i+1}: {query}")
            print("Top Semantic Result:", results[0][:100], "...\n")
            print("Expected:", self.expected_answers[i], "\n")

        print("=== Hybrid Retriever ===")
        for i, query in enumerate(self.queries):
            results = self.hybrid_retrieve(query)
            print(f"Query {i+1}: {query}")
            print("Top Hybrid Result:", results[0][:100], "...\n")
            print("Expected:", self.expected_answers[i], "\n")
# Usage Example
retriever_eval = retreiver_evaluation(docs, queries, Expected_answers)
retriever_eval.evaluate()


=== TF-IDF Retriever ===
Query 1: When was the Constitution of Nepal officially published?
Top Result: 1 
 
 
 
 
 
THE CONSTITUTION OF NEPAL ...

Expected: The Constitution of Nepal was officially published on September 20, 2015 (2072.6.3). 

Query 2: What is the Constitutional rule regarding the death penalty in Nepal?
Top Result: 199 
 
(2) 
While making a recommendation for appointment to the office of 
the Chief Justice, the C ...

Expected: The Constitution strictly prohibits the making of any law that provides for the death penalty. 

Query 3: How is a 'heinous offence' defined in Nepal's criminal law?
Top Result: 17 
Chapter -3 
Criminal Conspiracy, Attempt, Abetment and Accomplice 
33. Prohibition of criminal c ...

Expected: A heinous offence is an act punishable by life imprisonment or a prison term of more than ten years. 

Query 4: What is the minimum age for a child to be held criminally?
Top Result: 25 
 
 
 
(2) 
Every woman shall have the right to safe motherhood and 


# Retriever Evaluation Metrics



In [ ]:
#  evaluates TF-IDF, Semantic, and Hybrid retrievers using keyword-based metrics.
import re

def extract_keywords(text):
    # Remove punctuation and split into words
    words = re.findall(r'\b\w+\b', text.lower())
    # Remove common stopwords for better matching
    stopwords = set(['the', 'is', 'a', 'an', 'of', 'and', 'to', 'in', 'for', 'by', 'on', 'with', 'as', 'at', 'from', 'that', 'this', 'it', 'be', 'or', 'are', 'was', 'were', 'which', 'who', 'can', 'may'])
    return [w for w in words if w not in stopwords]

def keyword_match(expected, result, min_overlap=0.5):
    # Compute keyword overlap between expected answer and retrieved result
    expected_keywords = set(extract_keywords(expected))
    result_keywords = set(extract_keywords(result))
    if not expected_keywords:
        return False
    overlap = len(expected_keywords & result_keywords) / len(expected_keywords)
    return overlap >= min_overlap

def compute_metrics_keywords(retrieve_fn, queries, expected_answers, k=3, min_overlap=0.5):
    # Evaluate retriever performance using keyword overlap
    top1_hits = 0
    topk_hits = 0
    reciprocal_ranks = []
    for i, query in enumerate(queries):
        results = retrieve_fn(query, top_k=k)
        expected = expected_answers[i]
        found = False
        rank = None
        for idx, res in enumerate(results):
            if keyword_match(expected, res, min_overlap):
                found = True
                rank = idx + 1
                break
        if found:
            if rank == 1:
                top1_hits += 1
            topk_hits += 1
            reciprocal_ranks.append(1.0 / rank)
        else:
            reciprocal_ranks.append(0.0)
    n = len(queries)
    top1_rate = top1_hits / n
    topk_rate = topk_hits / n
    mrr = sum(reciprocal_ranks) / n
    return top1_rate, topk_rate, mrr

# Define retrievers to evaluate (Dense retriever removed)
retrievers = {
    'TF-IDF': tfidf_retrieve,
    'Semantic': semantic_retrieve,
    'Hybrid': hybrid_retrieve
}

print("\n=== Retriever Evaluation Metrics ===")
for name, fn in retrievers.items():
    # Print Top-1, Top-3 hit rates and MRR for each retriever
    top1, top3, mrr = compute_metrics_keywords(fn, queries, Expected_answers, k=3, min_overlap=0.5)
    print(f"{name} Retriever:")
    print(f"  Top-1 Hit Rate: {top1:.2f}")
    print(f"  Top-3 Hit Rate: {top3:.2f}")
    print(f"  MRR: {mrr:.2f}\n")


=== Retriver Evaluation Metrics ===
TF-IDF Retriever:
  Top-1 Hit Rate: 0.25
  Top-3 Hit Rate: 0.50
  MRR: 0.38

Semantic Retriever:
  Top-1 Hit Rate: 0.38
  Top-3 Hit Rate: 0.62
  MRR: 0.50

Hybrid Retriever:
  Top-1 Hit Rate: 0.25
  Top-3 Hit Rate: 0.62
  MRR: 0.42

Semantic Retriever:
  Top-1 Hit Rate: 0.38
  Top-3 Hit Rate: 0.62
  MRR: 0.50

Hybrid Retriever:
  Top-1 Hit Rate: 0.25
  Top-3 Hit Rate: 0.62
  MRR: 0.42

